In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import io
from PIL import Image
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from operator import itemgetter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [2]:
class Config:
    BASE_PATH = 'isic-2024-challenge/'
    TRAIN_IMAGE_PATH = 'train-image.hdf5'
    TRAIN_METADATA_PATH = 'train-metadata.csv'
    TEST_IMAGE_PATH = 'test-image.hdf5'
    TEST_METADATA_PATH = 'test-metadata.csv'
    
    # Data processing
    IMAGE_SIZE = (120, 120)
    VALIDATION_SPLIT = 0.1
    RANDOM_STATE = 42
    
    BATCH_SIZE = 32
    
    class MetadataModule:
        ACTIVATION = 'relu'
        KERNEL_INITIALIZER = 'he_normal'
        
    class ImageModule:
        ACTIVATION = 'relu'
        KERNEL_INITIALIZER = 'he_normal'

# Preprocesamiento

In [3]:
train_hdf5 = h5py.File(Config.BASE_PATH + Config.TRAIN_IMAGE_PATH, 'r')
test_hdf5 = h5py.File(Config.BASE_PATH + Config.TEST_IMAGE_PATH, 'r')

train_metadata = pd.read_csv(Config.BASE_PATH + Config.TRAIN_METADATA_PATH)
test_metadata = pd.read_csv(Config.BASE_PATH + Config.TEST_METADATA_PATH)

train_fnames = train_metadata["isic_id"].tolist()
test_fnames = test_metadata["isic_id"].tolist()

train_target = train_metadata["target"]

split = StratifiedShuffleSplit(n_splits=1, test_size=Config.VALIDATION_SPLIT, random_state=Config.RANDOM_STATE)
for train_index, val_index in split.split(train_metadata, train_target):
    val_fnames = itemgetter(*val_index)(train_fnames)
    train_fnames = itemgetter(*train_index)(train_fnames)
    X_metadata_train, X_metadata_val = train_metadata.iloc[train_index], train_metadata.iloc[val_index]
    y_train, y_val = train_target.iloc[train_index], train_target.iloc[val_index]

/var/folders/bw/_7s8wxw93cngpt5f7bg5s3hm0000gn/T/ipykernel_19537/2340496625.py:4: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata = pd.read_csv(Config.BASE_PATH + Config.TRAIN_METADATA_PATH)


In [4]:
only_train_cols = ["target", "lesion_id", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "mel_mitotic_index", "mel_thick_mm", "tbp_lv_dnn_lesion_confidence"]
unuseful_cols = ["image_type", "patient_id"]
removable_cols = only_train_cols + unuseful_cols + ["isic_id"]

numeric_features = train_metadata.select_dtypes(include=['float64', 'int64']).columns.difference(removable_cols)
cat_features = train_metadata.select_dtypes(include=['object']).columns.difference(removable_cols)

numeric_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', StandardScaler())
])

cat_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('cat', cat_pipeline, cat_features)
    ])

metadata_preprocessing_pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

X_train_metadata_preprocessed = metadata_preprocessing_pipeline.fit_transform(X_metadata_train)
X_val_metadata_preprocessed = metadata_preprocessing_pipeline.transform(X_metadata_val)
X_test_metadata_preprocessed = metadata_preprocessing_pipeline.transform(test_metadata)


In [5]:
# TRAIN
train_target_ds = tf.data.Dataset.from_tensor_slices(y_train)

def load_train_image(id):
    image = Image.open(io.BytesIO(np.array(train_hdf5[id.numpy()])))
    image = np.array(image.resize(Config.IMAGE_SIZE)).reshape(120, 120, 3)
    return image

def load_test_image(id):
    image = Image.open(io.BytesIO(np.array(test_hdf5[id.numpy()])))
    image = np.array(image.resize(Config.IMAGE_SIZE)).reshape(120, 120, 3)
    return image

def set_shapes(image):
    image.set_shape([120, 120, 3])
    return image

# Create a dataset for images
train_image_ds = tf.data.Dataset.from_tensor_slices(tf.constant(train_fnames))
train_image_ds = train_image_ds.map(lambda x: tf.py_function(load_train_image, [x], tf.float32))
train_image_ds = train_image_ds.map(set_shapes)
train_solo_image_ds = tf.data.Dataset.zip((train_image_ds, train_target_ds)).batch(Config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Create a dataset for metadata
train_metadata_ds = tf.data.Dataset.from_tensor_slices(X_train_metadata_preprocessed)
train_solo_metadata_ds = tf.data.Dataset.zip((train_metadata_ds, train_target_ds)).batch(Config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Combine the datasets
train_ds = tf.data.Dataset.zip(((train_image_ds, train_metadata_ds), train_target_ds))
train_ds = train_ds.shuffle(1000).batch(Config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# VAL
val_target_ds = tf.data.Dataset.from_tensor_slices(y_val)

# Create a dataset for images
val_image_ds = tf.data.Dataset.from_tensor_slices(tf.constant(val_fnames))
val_image_ds = val_image_ds.map(lambda x: tf.py_function(load_train_image, [x], tf.float32))
val_image_ds = val_image_ds.map(set_shapes)
val_solo_image_ds = tf.data.Dataset.zip((val_image_ds, val_target_ds)).batch(Config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Create a dataset for metadata
val_metadata_ds = tf.data.Dataset.from_tensor_slices(X_val_metadata_preprocessed)
val_solo_metadata_ds = tf.data.Dataset.zip((val_metadata_ds, val_target_ds)).batch(Config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Combine the datasets
val_ds = tf.data.Dataset.zip(((val_image_ds, val_metadata_ds), val_target_ds))
val_ds = val_ds.batch(Config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# TEST
test_image_ds = tf.data.Dataset.from_tensor_slices(tf.constant(test_fnames))
test_image_ds = test_image_ds.map(lambda x: tf.py_function(load_test_image, [x], tf.float32))
test_image_ds = test_image_ds.map(set_shapes)

test_metadata_ds = tf.data.Dataset.from_tensor_slices(X_test_metadata_preprocessed)
test_ds = tf.data.Dataset.zip((test_image_ds, test_metadata_ds)).batch(Config.BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Auxiliary functions and classes

In [6]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

def pauc_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    v_gt = abs(y_true - 1)
    v_pred = 1.0 - y_pred
    min_tpr = 0.80
    max_fpr = 1 - min_tpr
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return partial_auc

class PAUCCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data, batch_size):
        super(PAUCCallback, self).__init__()
        self.validation_data = validation_data
        self.batch_size = batch_size

    def on_epoch_end(self, epoch, logs=None):
        # Get predictions for validation data
        val_pred = self.model.predict(self.validation_data, verbose=0)
        
        # Extract true labels from validation data
        y_val = np.concatenate([y for x, y in self.validation_data], axis=0)
        
        # Calculate pAUC score
        pauc = pauc_score(y_val, val_pred)
        
        # Optionally, you can add the pAUC score to the logs
        logs['val_pauc'] = pauc

# Metadata module

In [7]:
metadata_input_shape = next(iter(train_metadata_ds.take(1))).shape

metadata_model = Sequential([
    Dense(64, activation=Config.MetadataModule.ACTIVATION, kernel_initializer=Config.MetadataModule.KERNEL_INITIALIZER, input_shape=metadata_input_shape),
    Dense(32, activation=Config.MetadataModule.ACTIVATION, kernel_initializer=Config.MetadataModule.KERNEL_INITIALIZER),
    Dense(1, activation='sigmoid')
])

# Compile the model
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002)
metadata_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
pauc_callback = PAUCCallback(val_solo_metadata_ds, Config.BATCH_SIZE)

# Display model summary
metadata_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                5312      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 7425 (29.00 KB)
Trainable params: 7425 (29.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
metadata_model.fit(train_solo_metadata_ds, validation_data=val_solo_metadata_ds, epochs=1, callbacks=[pauc_callback])

11280/11280 [==============================] - 5s 455us/step - loss: 0.0105 - accuracy: 0.9985 - val_loss: 0.0062 - val_accuracy: 0.9990 - val_pauc: 0.1245


# Image Module

In [9]:
image_input_shape = next(iter(train_image_ds.take(1))).shape

image_model = Sequential([
    Conv2D(32, 3, 2, activation=Config.MetadataModule.ACTIVATION, kernel_initializer=Config.MetadataModule.KERNEL_INITIALIZER, input_shape=image_input_shape),
    Conv2D(16, 3, 2, activation=Config.MetadataModule.ACTIVATION, kernel_initializer=Config.MetadataModule.KERNEL_INITIALIZER),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation=Config.MetadataModule.ACTIVATION, kernel_initializer=Config.MetadataModule.KERNEL_INITIALIZER),
    Dense(1, activation='sigmoid')
])

# Compile the model
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
image_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
pauc_callback = PAUCCallback(val_solo_image_ds.take(100), Config.BATCH_SIZE)

# Display model summary
image_model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 59, 59, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 16)        4624      
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 16)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense_3 (Dense)             (None, 64)                200768    
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                      

In [10]:
image_model.fit(train_solo_image_ds.take(500), validation_data=val_solo_image_ds.take(100), epochs=1, callbacks=[pauc_callback])

500/500 [==============================] - 20s 40ms/step - loss: 0.8098 - accuracy: 0.9961 - val_loss: 0.7174 - val_accuracy: 0.9991 - val_pauc: 0.0200


# Combined Modules

In [11]:
image_input = tf.keras.Input(shape=image_input_shape)
metadata_input = tf.keras.Input(shape=metadata_input_shape)

# Clone and freeze image model layers
x_image = image_input
for layer in image_model.layers[:-1]:  # Exclude the last layer
    x_image = layer(x_image)
    layer.trainable = False

# Clone and freeze metadata model layers
x_metadata = metadata_input
for layer in metadata_model.layers[:-1]:  # Exclude the last layer
    x_metadata = layer(x_metadata)
    layer.trainable = False

# Concatenate the outputs of both models
combined = tf.keras.layers.Concatenate()([x_image, x_metadata])
x = tf.keras.layers.Dense(16, activation=Config.MetadataModule.ACTIVATION, kernel_initializer=Config.MetadataModule.KERNEL_INITIALIZER)(combined)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Define inputs
input = [image_input, metadata_input]

combined_model = tf.keras.Model(inputs=input, outputs=x)

pauc_callback = PAUCCallback(val_ds.take(100), Config.BATCH_SIZE)

# Compile the model
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002)
combined_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
combined_model.fit(train_ds.take(500), validation_data=val_ds.take(100), epochs=1, callbacks=[pauc_callback])

# Unfreeze layers in the image model
for layer in combined_model.layers:
    if isinstance(layer, tf.keras.Model) and layer.name == image_model.name:
        for sub_layer in layer.layers:
            sub_layer.trainable = True

# Unfreeze layers in the metadata model
for layer in combined_model.layers:
    if isinstance(layer, tf.keras.Model) and layer.name == metadata_model.name:
        for sub_layer in layer.layers:
            sub_layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.00001)
combined_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

print("Layers unfrozen and model recompiled for fine-tuning.")

combined_model.fit(train_ds.take(500), validation_data=val_ds.take(100), epochs=1, callbacks=[pauc_callback])

500/500 [==============================] - 17s 32ms/step - loss: 0.0380 - accuracy: 0.9984 - val_loss: 0.0550 - val_accuracy: 0.9991 - val_pauc: 0.0200
Layers unfrozen and model recompiled for fine-tuning.
500/500 [==============================] - 15s 29ms/step - loss: 0.0408 - accuracy: 0.9991 - val_loss: 0.0479 - val_accuracy: 0.9991 - val_pauc: 0.0199


In [18]:
submission = pd.read_csv(Config.BASE_PATH + 'sample_submission.csv')
# Create a dataset of zeros with the same length as the test dataset
zero_labels = tf.data.Dataset.from_tensor_slices(tf.zeros(len(test_ds)))

# Combine the test dataset with the zero labels
test_ds_with_zeros = tf.data.Dataset.zip((test_ds, zero_labels))

submission["target"] = combined_model.predict(test_ds_with_zeros)
submission.to_csv('submission.csv', index=False)

1/1 [==============================] - 0s 15ms/step
